In [3]:
import numpy as np, pandas as pd, matplotlib.pyplot as plt, tensorflow as tf
import sys, time
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, Dropout, MaxPooling1D, GlobalAveragePooling1D, Conv1DTranspose, LSTM, Dense, Flatten
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from sklearn.model_selection import train_test_split

In [4]:
from tensorflow.keras import backend as K

def r_squared(y_true, y_pred):
    ss_res = K.sum(K.square(y_true - y_pred))
    ss_tot = K.sum(K.square(y_true - K.mean(y_true)))
    return (1 - ss_res/(ss_tot + K.epsilon()))

In [5]:
base_data_train, base_data_test = np.load('../../data/training_data/training_data_1month.npy', allow_pickle=True)

tf.random.set_seed(42)

<h2> Scale Data </h2>

In [6]:
from sklearn.preprocessing import MinMaxScaler

base_data_train, base_data_test = np.load('../../data/training_data/training_data_1month.npy', allow_pickle=True)
scalers = {var_name: MinMaxScaler(feature_range=(-1,1)) for var_name in ['G.air.T', 'G.E_th_I', 'G.sky.T', 'G.E_el_I']}

air_var, sky_var, el_var, th_var = base_data_train[:,:,0], base_data_train[:,:,1], base_data_train[:,:,2], base_data_train[:,:,3]
air_var_test, sky_var_test, el_var_test, th_var_test = base_data_test[:,:,0], base_data_test[:,:,1], base_data_test[:,:,2], base_data_test[:,:,3]

scaled_data_train = np.stack((scalers['G.air.T'].fit_transform(air_var),
                             scalers['G.sky.T'].fit_transform(sky_var),
                             scalers['G.E_el_I'].fit_transform(el_var),
                             scalers['G.E_th_I'].fit_transform(th_var)), axis=-1)
scaled_data_test = np.stack((scalers['G.air.T'].fit_transform(air_var),
                             scalers['G.sky.T'].fit_transform(sky_var),
                             scalers['G.E_el_I'].fit_transform(el_var),
                             scalers['G.E_th_I'].fit_transform(th_var)), axis=-1)
print(scaled_data_train.shape, scaled_data_test.shape)

(108, 730, 4) (108, 730, 4)


In [7]:
def split_base_data(base_data_train, base_data_test, th_indices=[0, 3], el_indices=[1, 2]):
    th_base_data_train, th_base_data_test = base_data_train[:, :, th_indices], base_data_test[:, :, th_indices]
    el_base_data_train, el_base_data_test = base_data_train[:, :, el_indices], base_data_test[:, :, el_indices]
    return th_base_data_train, th_base_data_test, el_base_data_train, el_base_data_test

th_base_data_train, th_base_data_test, el_base_data_train, el_base_data_test = split_base_data(base_data_train, base_data_test)
print(th_base_data_train.shape, th_base_data_test.shape, el_base_data_train.shape, el_base_data_test.shape)

(108, 730, 2) (12, 730, 2) (108, 730, 2) (12, 730, 2)


<h2> CNN Models</h2>

In [8]:
def create_cnn(input_shape):
    model = Sequential([
        Conv1D(filters=32, kernel_size=24, activation='relu', input_shape=(input_shape)),
        MaxPooling1D(pool_size=2),
        Dropout(0.25),
        Conv1D(filters=64, kernel_size=24, activation='relu'),
        MaxPooling1D(pool_size=2),
        Dropout(0.25),
        Flatten(),
        Dense(64, activation='relu'),
        Dense(730, activation='linear')
    ])
    return model

In [9]:
def test_cnn(training_data, th_or_el):
    X_train = training_data[:,:,0].reshape(-1,730,1)
    y_train = training_data[:,:,1]

    if th_or_el == 0:
        X_test = scaled_data_test[:,:,0].reshape(-1, 730, 1)  
        y_test = scaled_data_test[:,:,3]
    else:
        X_test = scaled_data_test[:,:,1].reshape(-1, 730, 1)  
        y_test = scaled_data_test[:,:,2]

    X_train, X_train_val, y_train, y_train_val = train_test_split(X_train, y_train, test_size=0.2, shuffle=True)
                                                                  
    model = create_cnn((X_train.shape[1], X_train.shape[2]))
    model.compile(optimizer=Adam(), loss='mse', metrics=['mse', 'mae'])

    early_stopping = EarlyStopping(monitor='mse', patience=10, verbose=1, restore_best_weights=True)
    reduce_lr = ReduceLROnPlateau(monitor='mse', factor=0.5, patience=5, verbose=1)
    model.fit(X_train, y_train, epochs=100, batch_size=16, callbacks=[early_stopping, reduce_lr], verbose=0, validation_data=(X_train_val, y_train_val))

    loss, mse, mae = model.evaluate(X_test, y_test)
    r2 = r_squared(tf.convert_to_tensor(y_test, dtype=tf.float32), tf.convert_to_tensor(model.predict(X_test), dtype=tf.float32))
    
    return {'mse':mse, 'mae':mae, 'r2':r2.numpy()}


In [10]:
num_reps = 10

def test_cnn_wrapper(data, th_or_el=0):
    mse, mae, r2 = 0, 0, 0

    # Run each CNN training 10 times to ensure results are significant and not outliers
    for i in range(num_reps):
        print(f'RUN: {i}')
        results = test_cnn(np.random.permutation(data), th_or_el) # permuting the data for each run just to ensure full shuffling
        mse += results['mse']
        mae += results['mae']
        r2  += results['r2']

    return {'mse':mse/num_reps, 'mae':mae/num_reps, 'r2':r2/num_reps}
        

<h2> Load data </h2>

In [11]:
def load_data(base_path, context, dataset_names):
    datasets = {}
    for name in dataset_names:
        file_path = f'{base_path}/{context}_{name}_generated_samples.npy'
        datasets[name] = np.load(file_path, allow_pickle=True)
    return datasets

base_path_vae = '../../data/vae_synthetic_data/'
base_path_gan = '../../data/gan_synthetic_data/'
th_context = 'th_v_air'
el_context = 'el_v_sky'

th_vae_datasets = ['b20l5', 'b4l10', 'b16l10', 'b8l20', 'b4l5', 'b16l5', 'b20l3', 'b24l15', 'b8l15', 'b24l50']
th_gan_datasets = ['b32e500', 'b8e1000', 'b20e100', 'b32e100', 'b6e100', 'b8e500']

th_vae_data = load_data(base_path_vae, th_context, th_vae_datasets)
th_gan_data = load_data(base_path_gan, th_context, th_gan_datasets)

el_vae_datasets = ['b4l3', 'b8l10', 'b20l3', 'b20l20', 'b24l3', 'b32l3', 'b32l5']
el_gan_datasets = ['b4e1000','b6e100','b8e500','b8e1000', 'b10e100', 'b12e100', 'b12e500', 'b16e500', 'b20e500', 'b24e1000', 'b32e1000']

el_vae_data = load_data(base_path_vae, el_context, el_vae_datasets)
el_gan_data = load_data(base_path_gan, el_context, el_gan_datasets)

In [12]:
print(len(th_vae_data), [th_vae_data[th_vae_datasets[i]].shape for i in range(len(th_vae_data))])
print(len(th_gan_data), [th_gan_data[th_gan_datasets[i]].shape for i in range(len(th_gan_data))])

print(len(el_vae_data), [el_vae_data[el_vae_datasets[i]].shape for i in range(len(el_vae_data))])
print(len(el_gan_data), [el_gan_data[el_gan_datasets[i]].shape for i in range(len(el_gan_data))])

10 [(1000, 730, 2), (1000, 730, 2), (1000, 730, 2), (1000, 730, 2), (1000, 730, 2), (1000, 730, 2), (1000, 730, 2), (1000, 730, 2), (1000, 730, 2), (1000, 730, 2)]
6 [(1000, 730, 2), (1000, 730, 2), (1000, 730, 2), (1000, 730, 2), (1000, 730, 2), (1000, 730, 2)]
7 [(1000, 730, 2), (1000, 730, 2), (1000, 730, 2), (1000, 730, 2), (1000, 730, 2), (1000, 730, 2), (1000, 730, 2)]
11 [(1000, 730, 2), (1000, 730, 2), (1000, 730, 2), (1000, 730, 2), (1000, 730, 2), (1000, 730, 2), (1000, 730, 2), (1000, 730, 2), (1000, 730, 2), (1000, 730, 2), (1000, 730, 2)]


<h2> Train CNNs </h2>

<h3> Ground Truth Datasets </h3>

In [ ]:
pd.DataFrame(test_cnn_wrapper(scaled_data_train[:,:,[0,3]], 0), index=[0]).to_csv('../../data/evaluation_results/th_gt.csv')
pd.DataFrame(test_cnn_wrapper(scaled_data_train[:,:,[1,2]], 1), index=[0]).to_csv('../../data/evaluation_results/el_gt.csv')

<h3> Synthetic Datasets </h3>

In [ ]:
def evaluate_and_save_cnn(data_dicts, model_type, column_index, base_path='../../data/evaluation_results/'):
    for dataset_name, dataset in data_dicts.items():
        cnn_test_results = test_cnn_wrapper(dataset[0:216, :, :], column_index)

        result_df = pd.DataFrame(cnn_test_results, index=[0])
        result_file_name = f'{base_path}{model_type}_{"th" if column_index==0 else "el"}_{dataset_name}.csv'
        result_df.to_csv(result_file_name)
        print(f'CNN results for {dataset_name}: {result_df}')
        print(f'Saved CNN evaluation results for {dataset_name} to: {result_file_name}')


In [ ]:
evaluate_and_save_cnn(th_vae_data, 'vae', 0)
evaluate_and_save_cnn(th_gan_data, 'gan', 0)


In [ ]:
evaluate_and_save_cnn(el_vae_data, 'vae', 1)
evaluate_and_save_cnn(el_gan_data, 'gan', 1)


<h3> Blended Datasets </h3>

In [94]:
def evaluate_and_save_cnn_blended(data_dicts, model_type, column_index, base_path='../../data/evaluation_results/'):
    for dataset_name, dataset in data_dicts.items():
        scaled_data = scaled_data_train[:,:,[0,3]] if column_index==0 else scaled_data_train[:,:,[1,2]]
        cnn_test_results = test_cnn_wrapper(np.concatenate((scaled_data, dataset[0:216,:,:]), axis=0), column_index)

        result_df = pd.DataFrame(cnn_test_results, index=[0])
        result_file_name = f'{base_path}{model_type}_blended_{"th" if column_index==0 else "el"}_{dataset_name}.csv'
        result_df.to_csv(result_file_name)
        print(f'CNN results for {dataset_name}: {result_df}')
        print(f'Saved CNN evaluation results for {dataset_name} to: {result_file_name}')


In [ ]:
evaluate_and_save_cnn_blended(th_vae_data, 'vae', 0)
evaluate_and_save_cnn_blended(th_gan_data, 'gan', 0)


In [97]:
evaluate_and_save_cnn_blended(el_vae_data, 'vae', 1)
evaluate_and_save_cnn_blended(el_gan_data, 'gan', 1)


RUN: 0
4/4 [==============================] - 0s 2ms/step
RUN: 1
4/4 [==============================] - 0s 2ms/step
RUN: 2

Epoch 97: ReduceLROnPlateau reducing learning rate to 0.0005000000237487257.
4/4 [==============================] - 0s 2ms/step
RUN: 3
4/4 [==============================] - 0s 2ms/step
RUN: 4

Epoch 98: ReduceLROnPlateau reducing learning rate to 0.0005000000237487257.
4/4 [==============================] - 0s 2ms/step
RUN: 5

Epoch 85: ReduceLROnPlateau reducing learning rate to 0.0005000000237487257.

Epoch 95: ReduceLROnPlateau reducing learning rate to 0.0002500000118743628.
4/4 [==============================] - 0s 2ms/step
RUN: 6

Epoch 98: ReduceLROnPlateau reducing learning rate to 0.0005000000237487257.
4/4 [==============================] - 0s 2ms/step
RUN: 7

Epoch 90: ReduceLROnPlateau reducing learning rate to 0.0005000000237487257.
4/4 [==============================] - 0s 2ms/step
RUN: 8
4/4 [==============================] - 0s 2ms/step
RUN: 9
4/4

4/4 [==============================] - 0s 2ms/step
RUN: 6

Epoch 89: ReduceLROnPlateau reducing learning rate to 0.0005000000237487257.
4/4 [==============================] - 0s 2ms/step
RUN: 7

Epoch 91: ReduceLROnPlateau reducing learning rate to 0.0005000000237487257.
4/4 [==============================] - 0s 2ms/step
RUN: 8

Epoch 95: ReduceLROnPlateau reducing learning rate to 0.0005000000237487257.
4/4 [==============================] - 0s 2ms/step
RUN: 9

Epoch 74: ReduceLROnPlateau reducing learning rate to 0.0005000000237487257.
4/4 [==============================] - 0s 2ms/step
CNN results for b20l20:         mse       mae        r2
0  0.322602  0.439988  0.537752
Saved CNN evaluation results for b20l20 to: ../../data/evaluation_results/vae_blended_el_b20l20.csv
RUN: 0

Epoch 82: ReduceLROnPlateau reducing learning rate to 0.0005000000237487257.

Epoch 90: ReduceLROnPlateau reducing learning rate to 0.0002500000118743628.
4/4 [==============================] - 0s 2ms/step
RUN

4/4 [==============================] - 0s 1ms/step
RUN: 1

Epoch 82: ReduceLROnPlateau reducing learning rate to 0.0005000000237487257.
4/4 [==============================] - 0s 2ms/step
RUN: 2

Epoch 98: ReduceLROnPlateau reducing learning rate to 0.0005000000237487257.
4/4 [==============================] - 0s 2ms/step
RUN: 3

Epoch 98: ReduceLROnPlateau reducing learning rate to 0.0005000000237487257.
4/4 [==============================] - 0s 2ms/step
RUN: 4

Epoch 84: ReduceLROnPlateau reducing learning rate to 0.0005000000237487257.
4/4 [==============================] - 0s 2ms/step
RUN: 5

Epoch 88: ReduceLROnPlateau reducing learning rate to 0.0005000000237487257.
4/4 [==============================] - 0s 2ms/step
RUN: 6

Epoch 80: ReduceLROnPlateau reducing learning rate to 0.0005000000237487257.
4/4 [==============================] - 0s 2ms/step
RUN: 7

Epoch 75: ReduceLROnPlateau reducing learning rate to 0.0005000000237487257.
4/4 [==============================] - 0s 2ms/st

4/4 [==============================] - 0s 2ms/step
RUN: 6
4/4 [==============================] - 0s 2ms/step
RUN: 7

Epoch 91: ReduceLROnPlateau reducing learning rate to 0.0005000000237487257.
4/4 [==============================] - 0s 2ms/step
RUN: 8

Epoch 68: ReduceLROnPlateau reducing learning rate to 0.0005000000237487257.
4/4 [==============================] - 0s 2ms/step
RUN: 9

Epoch 74: ReduceLROnPlateau reducing learning rate to 0.0005000000237487257.
4/4 [==============================] - 0s 2ms/step
CNN results for b8e1000:         mse       mae        r2
0  0.415755  0.513284  0.404276
Saved CNN evaluation results for b8e1000 to: ../../data/evaluation_results/gan_blended_el_b8e1000.csv
RUN: 0
4/4 [==============================] - 0s 2ms/step
RUN: 1

Epoch 64: ReduceLROnPlateau reducing learning rate to 0.0005000000237487257.

Epoch 95: ReduceLROnPlateau reducing learning rate to 0.0002500000118743628.
4/4 [==============================] - 0s 2ms/step
RUN: 2
4/4 [========

4/4 [==============================] - 0s 2ms/step
RUN: 1
4/4 [==============================] - 0s 2ms/step
RUN: 2
4/4 [==============================] - 0s 2ms/step
RUN: 3

Epoch 70: ReduceLROnPlateau reducing learning rate to 0.0005000000237487257.
4/4 [==============================] - 0s 2ms/step
RUN: 4

Epoch 91: ReduceLROnPlateau reducing learning rate to 0.0005000000237487257.
4/4 [==============================] - 0s 2ms/step
RUN: 5

Epoch 91: ReduceLROnPlateau reducing learning rate to 0.0005000000237487257.
4/4 [==============================] - 0s 2ms/step
RUN: 6
4/4 [==============================] - 0s 2ms/step
RUN: 7

Epoch 91: ReduceLROnPlateau reducing learning rate to 0.0005000000237487257.
4/4 [==============================] - 0s 2ms/step
RUN: 8

Epoch 91: ReduceLROnPlateau reducing learning rate to 0.0005000000237487257.
4/4 [==============================] - 0s 2ms/step
RUN: 9
4/4 [==============================] - 0s 2ms/step
CNN results for b16e500:         mse 

4/4 [==============================] - 0s 2ms/step
CNN results for b32e1000:         mse       mae        r2
0  0.412594  0.513886  0.408805
Saved CNN evaluation results for b32e1000 to: ../../data/evaluation_results/gan_blended_el_b32e1000.csv


<h2> Select and compare data </h2>

In [13]:
import glob

def find_best_model(prefix):
    folder_path = '../../data/evaluation_results/'

    file_paths = glob.glob(f'{folder_path}{prefix}*.csv')
    
    mse_vals = {}
    best_val, best_df, best_name = float('inf'), None, ''
    
    for file_path in file_paths:
        df = pd.read_csv(file_path)
        mse_val = df['mse'].iloc[0]
        mse_vals[file_path] = mse_val
        
        if mse_val < best_val:
            best_val, best_df = mse_val, df
            best_name = file_path.split('/')[-1]


    sorted_mse_vals = sorted(mse_vals.items(), key=lambda item: item[1])
    
    for name, value in sorted_mse_vals:
        print(name.split('/')[-1], value)
        
    print(f'\nDataFrame with lowest MSE ({best_name}):\n{best_df}\n')

In [14]:
print('Context: Thermal')
find_best_model('vae_th')
find_best_model('gan_th')

print('\nContext: Electrical')
find_best_model('vae_el')
find_best_model('gan_el')


Context: Thermal
vae_th_b20l3.csv 0.1190055929124355
vae_th_b8l15.csv 0.1265775240957737
vae_th_b16l5.csv 0.1314404964447021
evaluation_results\vae_th_b20l5.csv 0.1520956456661224
evaluation_results\vae_th_b24l15.csv 0.1523252636194229
evaluation_results\vae_th_b4l10.csv 0.1532680049538612
evaluation_results\vae_th_b24l50.csv 0.1778552532196045
evaluation_results\vae_th_b16l10.csv 0.1802604213356971
evaluation_results\vae_th_b8l20.csv 0.5562011629343033

DataFrame with lowest MSE (evaluation_results\vae_th_b20l3.csv):
   Unnamed: 0       mse       mae        r2
0           0  0.119006  0.262577  0.332094

evaluation_results\gan_th_b8e1000.csv 0.2068981036543846
evaluation_results\gan_th_b32e100.csv 0.2844437450170517
evaluation_results\gan_th_b8e500.csv 0.2867301911115646
evaluation_results\gan_th_b20e100.csv 0.3444884002208709
evaluation_results\gan_th_b6e100.csv 0.4662655055522918
evaluation_results\gan_th_b32e500.csv 0.5470245212316514

DataFrame with lowest MSE (evaluation_results\

In [15]:
print('Context: Thermal')
find_best_model('vae_blended_th')
find_best_model('gan_blended_th')

print('\nContext: Electrical')
find_best_model('vae_blended_el')
find_best_model('gan_blended_el')

Context: Thermal
evaluation_results\vae_blended_th_b20l3.csv 0.0781151488423347
evaluation_results\vae_blended_th_b4l10.csv 0.0924433469772338
evaluation_results\vae_blended_th_b8l15.csv 0.0932417020201683
evaluation_results\vae_blended_th_b4l5.csv 0.0968162976205349
evaluation_results\vae_blended_th_b24l15.csv 0.09766411408782
evaluation_results\vae_blended_th_b16l5.csv 0.1016411677002906
evaluation_results\vae_blended_th_b20l5.csv 0.1065429620444774
evaluation_results\vae_blended_th_b24l50.csv 0.1090219363570213
evaluation_results\vae_blended_th_b16l10.csv 0.120886443555355
evaluation_results\vae_blended_th_b8l20.csv 0.1320243626832962

DataFrame with lowest MSE (evaluation_results\vae_blended_th_b20l3.csv):
   Unnamed: 0       mse       mae        r2
0           0  0.078115  0.203689  0.561587

evaluation_results\gan_blended_th_b20e100.csv 0.0895717576146125
evaluation_results\gan_blended_th_b32e500.csv 0.1010843776166439
evaluation_results\gan_blended_th_b8e1000.csv 0.1029256179928